# Overview of the dataset

The Aqar Dataset was made based on 4 major cities in Saudi Arabia:

- Riyadh
- Jeddah
- Dammam
- Khobar

The dataset is composed of over 3000 observations with 24 features as follows:

Feature| Description
---|----|
city| city where house locate in
district| district where house locate in
front| What is the house front is north, west .. etc
size| size in m^2
propertyage| property age for the house 
bedrooms| number of bedrooms 
bathrooms| number of bathrooms 
livingrooms| number of livingrooms 
kitchen| show whether the house have a kitchen or not garage| show whether the house have a garage or not driverroom| show whether the house have a driverroom or not maidroom| show whether the house have a maid_room or not
furnished| show whether the house is furnished or not
ac| show whether the house have a ac or not
roof| show whether the house have a space for roof on top or not
pool| show whether the house have a pool or not
frontyard| show whether the house have a frontyard or not
basement| show whether the house have a basement or not
duplex| show whether the house is a duplex or not
stairs| show whether the house have a stairs or not
elevator| show whether the house have an elevator or not
fireplace| show whether the house have a fireplace or not
price| show the price of the house
details| shows any additional details from the house owner about the house


In [ ]:
#| echo: false

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# support arabic plotting
import arabic_reshaper # pip install arabic-reshaper
from bidi.algorithm import get_display # pip install python-bidi

data = pd.read_csv("SA_Aqar.csv")

# make a function to plot arabic labels
def plot_arabic(labelsSeries: pd.Series):
    """
    A function that plots arabic literals in their required format instead of just floating letters
    \nArgs:
     - labelsSeries: a series of arabic labels 
    \nReturns:
     - result: a list of properly formatted arabic labels
    """

    # apply the reshaping method
    arabicLabels = labelsSeries.apply(arabic_reshaper.reshape)

    # initiate the formatting process
    result = []
    for label in arabicLabels:
        result.append(get_display(label))
    
    # return the formatted labels as a list if there are more than one value
    if len(result) == 1:
        return result[0]
    else:
        return result

# EDA Process

## Turki & Yousef

In [ ]:
#| label: Line Chart
#| fig-cap: Average price and size per city

d_yousef = data[data['size'] < 80000]
city_avg = d_yousef.groupby(plot_arabic(d_yousef['city'])).mean().sort_values('price', ascending = False)

ax1 = plt.subplot()
l1, = ax1.plot(city_avg['price'], color='red')
plt.ylabel(plot_arabic(pd.Series("السعر")))
ax2 = ax1.twinx()
l2, = ax2.plot(city_avg['size'], color='orange')

plt.ylabel(plot_arabic(pd.Series("المساحة")))
plt.legend([l1, l2], plot_arabic(pd.Series(["السعر", "المساحة"])))

<br><br>

In [ ]:
#| label: Side-by-side bar
#| fig-cap: Average price and size per distrit for Jeddah and Riyadh
from IPython import display

s = d_yousef.district.value_counts()
repeated = set(s[s > 30].index.values)
d_district = d_yousef[d_yousef.district.isin(repeated)]
'''
avg_size_dist = d_district.groupby(['city', 'district'])[['size', 'price']].mean().reset_index()
riyadh = avg_size_dist.query("city == 'الرياض' & size < 500")
jeddah = avg_size_dist.query("city == 'جدة' & price < 160000")

plt.figure(figsize=(10,10))

# RIYADH
x = np.arange(len(riyadh))
ax1 = plt.subplot(2,2,1)
w = 0.3
plt.title(plot_arabic(pd.Series('الرياض')))
plt.xticks(x + w /2, plot_arabic(riyadh['district']), rotation='60')
P =ax1.bar(x, riyadh['price'].sort_values(), width=w, color='r', align='center')
plt.ylabel(plot_arabic(pd.Series("السعر")))
ax2 = ax1.twinx()
S =ax2.bar(x + w, riyadh['size'].sort_values(), width=w,color='y',align='center')
ax2.axes.yaxis.set_visible(False)
plt.legend([P, S], plot_arabic(pd.Series(['متوسط السعر', 'متوسط المساحة'])))

# JEDDAH
x2 = np.arange(len(jeddah))
ax3 = plt.subplot(2,2,2)
w2 = 0.3
plt.title(plot_arabic(pd.Series('جدة')))
plt.xticks(x2 + w2 /2, plot_arabic(jeddah['district']), rotation='60')
P =ax3.bar(x2, jeddah['price'].sort_values(), width=w2, color='r', align='center')
ax3.axes.yaxis.set_visible(False)
ax4 = ax3.twinx()
S =ax4.bar(x2 + w2, jeddah['size'].sort_values(), width=w2,color='y',align='center')
plt.ylabel(plot_arabic(pd.Series("المساحة")))
plt.legend([P, S], plot_arabic(pd.Series(['متوسط السعر', 'متوسط المساحة'])))

plt.tight_layout()
plt.show()
'''
display.Image("./side-by-side-bar.png")

<br><br>

In [ ]:
#| label: Table
#| fig-cap: Highest average price districts

d_district.groupby(['city', 'district'])['price'].mean().sort_values(ascending=False).to_frame().head(10).reset_index()

## Ahmed

The following questions were in mind during this part of the EDA:

### Covariance: What is the factor that when present increases the price the most?


In [ ]:
#| label: corr
#| fig-cap: Covariance Heatmap
corr = data.corr()
fig, ax = plt.subplots(figsize=(8,5))
sns.heatmap(ax=ax, data= data.corr(), cmap="Greens");

As we can see in the @corr, we can observe that there are 4 major features that are considered higly correlated with price, these are:

- `driver_room`
- `pool`
- `ac`
- `basement`

Interesting...

How about a deep look at each feature with the price:


In [ ]:
#| label: featureplots
#| fig-cap: Visualizing the effect of the 4 features on the price
# see how great the above 4 feature affect the price
# creating the canvas
fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2,2, figsize=(15,10))

# plotting ax1 data: `driver_room`
ax1 = sns.barplot(ax=ax1, y=data["price"], x=data["driver_room"] ,data=data)

# configure the plot
ax1.set_title("Effect of Driver's room on price")
ax1.set_xlabel("")
labels = [item.get_text() for item in ax1.get_xticklabels()]
labels[0] = "Without Driver Room"
labels[1] = "With Driver Room"
ax1.set_xticklabels(labels)
ax1.set_ylabel("Price")


# plotting ax2 data: `ac`
ax2 = sns.barplot(ax=ax2, y=data["price"], x=data["ac"] ,data=data)

# configure the plot
ax2.set_title("Effect of AC on price")
ax2.set_xlabel("")
labels = [item.get_text() for item in ax2.get_xticklabels()]
labels[0] = "Without AC"
labels[1] = "With AC"
ax2.set_xticklabels(labels)
ax2.set_ylabel("Price")


# plotting ax3 data: `pool`
ax3 = sns.barplot(ax=ax3, y=data["price"], x=data["pool"] ,data=data)

# configure the plot
ax3.set_title("Effect of Pool on price")
ax3.set_xlabel("")
labels = [item.get_text() for item in ax3.get_xticklabels()]
labels[0] = "Without Pool"
labels[1] = "With Pool"
ax3.set_xticklabels(labels)
ax3.set_ylabel("Price")

# plotting ax4 data: `basement`
ax4 = sns.barplot(ax=ax4,y=data["price"], x=data["basement"] ,data=data)

# configure the plot
ax4.set_title("Effect of Basement on price")
ax4.set_xlabel("")
labels = [item.get_text() for item in ax4.get_xticklabels()]
labels[0] = "Without Basement"
labels[1] = "With Basement"
ax4.set_xticklabels(labels)
ax4.set_ylabel("Price");


We can conculde the following based on the @featureplots:

::: {.callout}
On average, villas with a basement room tend to be showcased at a higher rent rate.
:::

### Which has a higher impact on the rent prices: location or features?


In [ ]:
#| label: cityprice
#| fig-cap: Average price in each of the major cities
# find the price range in the 4 major cities
# create the canva
fig, ax = plt.subplots(figsize=(8,5))

# plot the data
sns.barplot(ax=ax ,y = "price", x=plot_arabic(data["city"]), data=data)

# configure the plot    
ax.set_title(plot_arabic(pd.Series("متوسط أسعار الأجار حسب المدينة")))
ax.set_ylabel(plot_arabic(pd.Series("الأسعار")))
ax.set_xlabel(plot_arabic(pd.Series("المدن")));

## Lana

### what frontage appears the most and what is the most desired

In [ ]:
plt.figure(figsize=(10,8))
plt.hist(x= plot_arabic(data['front']) , bins =20)
plt.show()

#insert price and frontage corr code

### what the ratio of bedrooms and living rooms to bathrooms and does it reflect the market demand 


In [ ]:
ratio = data[['bedrooms', 'bathrooms']].apply(lambda data: (data['bedrooms']/data['bathrooms']), axis=1)
ratio = ratio.mean()
ratio

### investigate rental units without kitchen


In [ ]:
nokit = data.groupby(['kitchen']).get_group(0)
nokit

sns.histplot(data=nokit, x="bedrooms", kde=True)

sns.histplot(data=nokit, x="bathrooms", kde=True, color="orange")

## Mohammed

The location of a house may be the most significant factor in determining the price, we don't have that in the dataset. Perhaps we can create this attribute.

In [ ]:
data['district'].head(10)

In [ ]:
# libraries to handel spacial data
import geopandas as gpd
import geopy

In [ ]:
#| echo: false
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium
from folium.plugins import FastMarkerCluster

Calling an API requesting the coordinates of the address we provide

In [ ]:
#|eval:false
# 0 - create a locator instance
locator = Nominatim(user_agent="myGeocoder")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.1)

# 2- - create location column
data['location'] = data['district'].apply(geocode)

# 3 - create longitude, laatitude and altitude from location column (returns tuple)
data['point'] = data['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# 4 - split point column into latitude, longitude and altitude columns
data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(data['point'].tolist(), index=data.index)

In [ ]:
#|echo:false
data = pd.read_csv("SA_Aqar_Added_Attributes.csv")
data['part_of_city'] = data['part_of_city'].astype('category')

In [ ]:
#|echo: true
data[['latitude', 'longitude', 'city']].head() 

In [ ]:
data['latitude'].isna().sum() / len(data['latitude'])

We need to process the data to turn it longitude and latitude (and city) into which part of the city a house belongs.

In [ ]:

def convert_coord(latitude: pd.Series, longitude: pd.Series, cities: pd.Series):
    """ given a latitude and longitude and cities, returns a single 
        attribute that encodes which part of the city a (latitude, longitude) 
        pair belongs to (e.g., north_east)"""

    middles = middle_of_cities(latitude, longitude, cities)
    part_of_city = assign_location(latitude, longitude, cities, middles)

    return part_of_city


def middle_of_cities(latitude: pd.Series, longitude: pd.Series, cities: pd.Series):
    """returns a dictionary containing four tuples where each tuple is 
        (latitude, longitude) of the middle of one of the cities """

    #store the produced middle values 
    middles = {}

    for city in cities.unique():
        indicies = cities == city
        relevent_city = cities.loc[indicies]
        relevent_latitude = latitude.loc[indicies]
        relevent_longitude = longitude.loc[indicies]

        # consider taking the trimmed median instead
        middle_latitude = relevent_latitude.median()
        middle_longitude = relevent_longitude.median()

        middles[city] = (middle_latitude, middle_longitude)

    return middles

def assign_location(latitude: pd.Series, longitude: pd.Series, cities: pd.Series, middle_of_cities: dict):
    """ creates a series of the same length as the original dataframe assigning each instance one of
        north_east, north_west, south_east, south_west depending on where the latitude and longitude lie
        relative to the relevent middle of city"""

    location_4 = []
   
    for i in range(len(latitude)):
        if latitude[i] is None:
            location_4.append(None)
            continue

        relevent_city_middle = middle_of_cities[cities[i]]

        if latitude[i] > relevent_city_middle[0]:
            # north
            if longitude[i] > relevent_city_middle[1]:
                # east
                location_4.append("شمال شرق")
            else:
                location_4.append("شمال غرب")
        else:
            # south
            if longitude[i] > relevent_city_middle[1]:
                # east
                location_4.append("جنوب شرق")
            else:
                location_4.append("جنوب غرب")

    return pd.Series(location_4)


Run the functions 

In [ ]:
#|echo: true
#|eval: false
data['part_of_city'] = convert_coord(data.latitude, data.longitude, data.city)

In [ ]:
#|echo: true
data[['city', 'part_of_city']].head(10)

Let's visualize some basic statistics 

In [ ]:
# Bar plot

grouped_city = data.groupby('city')
parts_order = pd.Series(['شمال شرق', 'شمال غرب', 'جنوب شرق', 'جنوب غرب'])

fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2,2, figsize=(12,9))

plt.suptitle("The Number of Houses in Each Side of The Cities")

sns.countplot(ax=ax1, data=grouped_city.get_group('الرياض'), x=plot_arabic(grouped_city.get_group('الرياض')['part_of_city']), order=plot_arabic(parts_order))
ax1.set_title("Riyadh")
ax1.set_ylabel("Number of Houses")

sns.countplot(ax=ax2, data=grouped_city.get_group('الخبر'), x=plot_arabic(grouped_city.get_group('الخبر')['part_of_city']), order=plot_arabic(parts_order))
ax2.set_title('Khubar')
ax2.set_ylabel("")

sns.countplot(ax=ax3, data=grouped_city.get_group('الدمام'), x=plot_arabic(grouped_city.get_group('الدمام')['part_of_city']), order=plot_arabic(parts_order))
ax3.set_title('Dammam')
ax3.set_ylabel("Number of Houses")

sns.countplot(ax=ax4, data=grouped_city.get_group('جدة'), x=plot_arabic(grouped_city.get_group('جدة')['part_of_city']), order=plot_arabic(parts_order))
ax4.set_title('Jeddah')
ax4.set_ylabel("")

The north east is the most expensive part of every city! (Except in Riyadh it's a close second.)

In [ ]:
#| echo: false
# Visualizing the mean price for each city and part within it

grouped_city = data.groupby('city')
# grouped_city.get_group('الرياض')
# grouped_city.get_group('الخبر')
# grouped_city.get_group('الدمام')
# grouped_city.get_group('جدة')
parts_order = pd.Series(['شمال شرق', 'شمال غرب', 'جنوب شرق', 'جنوب غرب'])

fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2,2, figsize=(12,9))

plt.suptitle("The Median Price: Excluding Outliers (Top and Bottom %5)")


sns.barplot(ax=ax1, data=grouped_city.get_group('الرياض'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('الرياض').groupby('part_of_city')['price'].mean()), order=plot_arabic(parts_order))
ax1.set_title("Riyadh")
ax1.set_ylabel("Mean Price")

sns.barplot(ax=ax2, data=grouped_city.get_group('الخبر'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('الخبر').groupby('part_of_city')['price'].mean()), order=plot_arabic(parts_order))
ax2.set_title('Khubar')
ax2.set_ylabel("")

sns.barplot(ax=ax3, data=grouped_city.get_group('الدمام'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('الدمام').groupby('part_of_city')['price'].mean()), order=plot_arabic(parts_order))
ax3.set_title('Dammam')
ax3.set_ylabel("Mean Price")

sns.barplot(ax=ax4, data=grouped_city.get_group('جدة'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('جدة').groupby('part_of_city')['price'].mean()), order=plot_arabic(parts_order))
ax4.set_title('Jeddah')
ax4.set_ylabel("")

In [ ]:
# Visualizing the median price for each city and part within it

grouped_city = data.groupby('city')

parts_order = pd.Series(['شمال شرق', 'شمال غرب', 'جنوب شرق', 'جنوب غرب'])

fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2,2, figsize=(12,9))

plt.suptitle("The Median Price: Complete Dataset")

sns.barplot(ax=ax1, data=grouped_city.get_group('الرياض'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('الرياض').groupby('part_of_city')['price'].median()), order=plot_arabic(parts_order))
ax1.set_title("Riyadh")
ax1.set_ylabel("Median Price")

sns.barplot(ax=ax2, data=grouped_city.get_group('الخبر'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('الخبر').groupby('part_of_city')['price'].median()), order=plot_arabic(parts_order))
ax2.set_title('Khubar')
ax2.set_ylabel("")

sns.barplot(ax=ax3, data=grouped_city.get_group('الدمام'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('الدمام').groupby('part_of_city')['price'].median()), order=plot_arabic(parts_order))
ax3.set_title('Dammam')
ax3.set_ylabel("Median Price")

sns.barplot(ax=ax4, data=grouped_city.get_group('جدة'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('جدة').groupby('part_of_city')['price'].median()), order=plot_arabic(parts_order))
ax4.set_title('Jeddah')
ax4.set_ylabel("")


In [ ]:
#| echo: false
# excluding the outlier values in price and size

price_percentile_95th = np.percentile(data["price"], 95)
size_percentile_95th = np.percentile(data["size"], 95)
price_percentile_5th = np.percentile(data["price"], 5)
size_percentile_5th = np.percentile(data["size"], 5)

exc_outliers = data.loc[data["price"] < price_percentile_95th]
exc_outliers = exc_outliers.loc[data["size"] < size_percentile_95th]
exc_outliers = exc_outliers.loc[data["price"] > price_percentile_5th]
exc_outliers = exc_outliers.loc[data["size"] > size_percentile_5th]

How would the plots look if we chose the trimmed median instead of the median?

In [ ]:
#| echo: false
# Visualizing the median price for each city and part within it (Excluding outliers in price and size)

grouped_city = exc_outliers.groupby('city')
# grouped_city.get_group('الرياض')
# grouped_city.get_group('الخبر')
# grouped_city.get_group('الدمام')
# grouped_city.get_group('جدة')
parts_order = pd.Series(['شمال شرق', 'شمال غرب', 'جنوب شرق', 'جنوب غرب'])

fig, ((ax1,ax2), (ax3,ax4)) = plt.subplots(2,2, figsize=(12,9))
plt.suptitle("The Median Price: Complete Dataset")


sns.barplot(ax=ax1, data=grouped_city.get_group('الرياض'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('الرياض').groupby('part_of_city')['price'].median()), order=plot_arabic(parts_order))
ax1.set_title("Riyadh")
ax1.set_ylabel("Avg. Price")

sns.barplot(ax=ax2, data=grouped_city.get_group('الخبر'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('الخبر').groupby('part_of_city')['price'].median()), order=plot_arabic(parts_order))
ax2.set_title('Khubar')
ax2.set_ylabel("")

sns.barplot(ax=ax3, data=grouped_city.get_group('الدمام'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('الدمام').groupby('part_of_city')['price'].median()), order=plot_arabic(parts_order))
ax3.set_title('Dammam')
ax3.set_ylabel("Avg. Price")

sns.barplot(ax=ax4, data=grouped_city.get_group('جدة'), x=plot_arabic(pd.Series(grouped_city.get_group('الرياض')['part_of_city'].cat.categories)), 
  y=pd.Series(grouped_city.get_group('جدة').groupby('part_of_city')['price'].median()), order=plot_arabic(parts_order))
ax4.set_title('Jeddah')
ax4.set_ylabel("")

Some cities have a greater variance in price than others.

In [ ]:
#| echo: false
# remove all the changes I made to the dataset
data = pd.read_csv("SA_Aqar.csv")

# Overall Conclusion

# Challenges Faced

During this activity with the dataset, we overcame such great obstacles that include:

* Plotting Arabic labels in the correct formatting
* Genearating mutliple plots and interacting them with Seaborn plots
* Conflicts with team working and collaboration